In [23]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, KFold

In [12]:
train = pd.read_csv("/storage/home/ndh5286/Fun Projects/Spaceship Titanic/train.csv")
test = pd.read_csv("/storage/home/ndh5286/Fun Projects/Spaceship Titanic/test.csv")

#Replace the strings with respective numbers
train['HomePlanet'] = pd.factorize(train['HomePlanet'])[0]
train['CryoSleep'] = pd.factorize(train['CryoSleep'])[0]
train['Cabin'] = pd.factorize(train['Cabin'])[0]
train['Destination'] = pd.factorize(train['Destination'])[0]
train['VIP'] = pd.factorize(train['VIP'])[0]

test['HomePlanet'] = pd.factorize(test['HomePlanet'])[0]
test['CryoSleep'] = pd.factorize(test['CryoSleep'])[0]
test['Cabin'] = pd.factorize(test['Cabin'])[0]
test['Destination'] = pd.factorize(test['Destination'])[0]
test['VIP'] = pd.factorize(test['VIP'])[0]

train.corr()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
HomePlanet,1.000000,-0.037065,-0.004802,-0.212843,-0.120503,-0.053798,0.178878,-0.279357,0.077910,-0.203228,-0.228913,-0.094689
CryoSleep,-0.037065,1.000000,-0.013912,0.090537,-0.062649,-0.050223,-0.226037,-0.191130,-0.190339,-0.182918,-0.179129,0.424362
Cabin,-0.004802,-0.013912,1.000000,-0.000167,-0.007797,0.001420,-0.003791,-0.009717,0.019949,-0.009816,0.015029,0.021866
Destination,-0.212843,0.090537,-0.000167,1.000000,0.004248,0.027978,-0.044733,0.110659,-0.022977,0.053614,0.073654,0.104817
Age,-0.120503,-0.062649,-0.007797,0.004248,1.000000,0.070953,0.068723,0.130421,0.033133,0.123970,0.101007,-0.075026
VIP,-0.053798,-0.050223,0.001420,0.027978,0.070953,1.000000,0.029853,0.089942,0.027081,0.048117,0.087033,-0.027802
RoomService,0.178878,-0.226037,-0.003791,-0.044733,0.068723,0.029853,1.000000,-0.015889,0.054480,0.010080,-0.019581,-0.244611
FoodCourt,-0.279357,-0.191130,-0.009717,0.110659,0.130421,0.089942,-0.015889,1.000000,-0.014228,0.221891,0.227995,0.046566
ShoppingMall,0.077910,-0.190339,0.019949,-0.022977,0.033133,0.027081,0.054480,-0.014228,1.000000,0.013879,-0.007322,0.010141
Spa,-0.203228,-0.182918,-0.009816,0.053614,0.123970,0.048117,0.010080,0.221891,0.013879,1.000000,0.153821,-0.221131


In [13]:
train = train.drop('ShoppingMall', axis = 1)   #Drop low correlated columns to prevent overfitting
train = train.drop('Cabin', axis = 1)
train = train.drop('VIP', axis = 1)
train = train.drop('Name', axis = 1)
train = train.drop('PassengerId', axis = 1)

test = test.drop('ShoppingMall', axis = 1)   #Drop low correlated columns to prevent overfitting
test = test.drop('Cabin', axis = 1)
test = test.drop('VIP', axis = 1)
test = test.drop('Name', axis = 1)
PassengerId = test['PassengerId']
test = test.drop('PassengerId', axis = 1)

#Impute with mean
train = train.fillna(train.mean())
test = test.fillna(test.mean())

#Assign dependent variable before scaling
y_train = train.iloc[:, 1]

In [14]:
#Initialize the StandardScaler on Train
scaler = StandardScaler()
scaler.fit(train)
scaled_data = scaler.transform(train)
train = pd.DataFrame(scaled_data, columns = train.columns)

#Initialize the StandardScaler on Test
scaler.fit(test)
scaled_data = scaler.transform(test)
test = pd.DataFrame(scaled_data, columns = test.columns)

#Assign independent variables
X_train = train.iloc[:, :-1]
X_test = test

In [5]:
#Creating a Logistic Regression model
lr = LogisticRegression()

#Training the model on the training data
lr.fit(X_train, y_train)

#Making predictions on the testing data
y_pred = lr.predict(X_test)
y_pred = pd.DataFrame(y_pred, columns = ["Transported"])

In [15]:
result = PassengerId
result = pd.concat([result, y_pred], axis=1)

#Replace zeros with FALSE and ones with TRUE in the second column (B)
result['Transported'] = result['Transported'].replace({1: False, 0: True})

#Write DataFrame to a CSV file
result.to_csv('result.csv', index=False)

In [7]:
print(y_train)

0       0
1       0
2       0
3       0
4       0
       ..
8688    0
8689    1
8690    0
8691    0
8692    0
Name: CryoSleep, Length: 8693, dtype: int64


In [29]:
#Necessary for whatever reason
le = LabelEncoder()
y_train = le.fit_transform(y_train)

xgb_model = xgb.XGBClassifier()

xgb_model.fit(X_train, y_train)

# Predict on the test set
y_pred = xgb_model.predict(X_test)
y_pred = pd.DataFrame(y_pred, columns = ["Transported"])

result = PassengerId
result = pd.concat([result, y_pred], axis=1)
result['Transported'] = result['Transported'].replace({1: False, 0: True})

#Replace zeros with FALSE and ones with TRUE in the second column (B)
#result['Transported'] = result['Transported'].replace({0: False, 1: True})
print(result)

#Write DataFrame to a CSV file
result.to_csv('result_xgb.csv', index=False)

     PassengerId  Transported
0        0013_01         True
1        0018_01        False
2        0019_01         True
3        0021_01        False
4        0023_01        False
...          ...          ...
4272     9266_02         True
4273     9269_01        False
4274     9271_01         True
4275     9273_01        False
4276     9277_01         True

[4277 rows x 2 columns]


[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
